In [ ]:
# [셀 2] 라이브러리/경로
import os
import pandas as pd
from sqlalchemy import create_engine, text
from getpass import getpass

CSV_PATH = "/mnt/data/bjd_first_by_sgg_nm.csv"  # 업로드된 파일 경로


In [ ]:
# [셀 3] CSV 로드 & 기본 점검
df = pd.read_csv(CSV_PATH)

print("shape:", df.shape)
print(df.dtypes)
display(df.head())

# 필수 컬럼 확인
required = {"bjd_cd", "center_long", "center_lati", "bjd_nm"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# 결측/중복 점검
print("null counts:\n", df.isna().sum())
print("duplicate bjd_cd:", df["bjd_cd"].duplicated().sum())


In [ ]:
# [셀 4] MySQL 접속정보 입력 (안전하게 getpass 사용)
MYSQL_HOST = os.getenv("MYSQL_HOST") or input("MYSQL_HOST (예: 127.0.0.1): ").strip()
MYSQL_PORT = int(os.getenv("MYSQL_PORT") or input("MYSQL_PORT (기본 3306): ") or "3306")
MYSQL_USER = os.getenv("MYSQL_USER") or input("MYSQL_USER: ").strip()
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD") or getpass("MYSQL_PASSWORD: ")
MYSQL_DB = os.getenv("MYSQL_DB") or input("MYSQL_DB (예: mydb): ").strip()

# charset=utf8mb4 권장 (한글)
engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
    f"?charset=utf8mb4",
    pool_pre_ping=True,
)


In [ ]:
# [셀 5] 테이블 생성 (PK 포함) - 권장 스키마
TABLE = "district_all"  # 원하는 테이블명으로 바꿔도 됨

create_sql = f"""
CREATE TABLE IF NOT EXISTS `{TABLE}` (
  `bjd_cd` BIGINT NOT NULL,
  `center_long` DOUBLE NULL,
  `center_lati` DOUBLE NULL,
  `bjd_nm` VARCHAR(100) NOT NULL,
  PRIMARY KEY (`bjd_cd`),
  INDEX `idx_bjd_nm` (`bjd_nm`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
"""

with engine.begin() as conn:
    conn.execute(text(create_sql))

print("table ready:", TABLE)


In [ ]:
# [셀 6] 적재 방식 선택
# 옵션 A) 완전 새로 넣기(테이블 비우고 insert)
# 옵션 B) upsert(동일 bjd_cd 있으면 update)

LOAD_MODE = "A"  # "A" 또는 "B"


In [ ]:
# [셀 7-A] 완전 새로 넣기(테이블 TRUNCATE 후 벌크 insert)
if LOAD_MODE.upper() == "A":
    with engine.begin() as conn:
        conn.execute(text(f"TRUNCATE TABLE `{TABLE}`;"))

    # pandas.to_sql은 PK/인덱스는 건드리지 않고, 데이터만 insert 함
    df.to_sql(TABLE, con=engine, if_exists="append", index=False, chunksize=5000, method="multi")
    print("loaded with TRUNCATE+append:", len(df))


In [ ]:
# [셀 8] 적재 검증 (행 수/샘플)
with engine.connect() as conn:
    n = conn.execute(text(f"SELECT COUNT(*) FROM `{TABLE}`;")).scalar_one()
    sample = conn.execute(text(f"SELECT * FROM `{TABLE}` ORDER BY bjd_cd LIMIT 5;")).fetchall()

print("row_count_in_db:", n)
print("sample_rows:")
for r in sample:
    print(r)


In [ ]:
# [셀 9] 자주 하는 조회 예시
# 1) 특정 시군구명 포함 검색
kw = "서울특별시"
with engine.connect() as conn:
    rows = conn.execute(
        text(f"SELECT * FROM `{TABLE}` WHERE bjd_nm LIKE :kw ORDER BY bjd_cd LIMIT 10;"),
        {"kw": f"%{kw}%"},
    ).fetchall()

for r in rows:
    print(r)
